In [25]:
import os
import itertools
import csv

In [26]:
# Crear directorio SiData si no existe
directorio = ".in"
os.makedirs(directorio, exist_ok=True)

# Definiendo parámetros de entrada

In [27]:
# Parámetros para iterar
elemento = 'Si' 
ecuts = range(10, 101,5)  # Energía de corte de 10 a 100 Ry
kpuntos = [(x, x, x) for x in range(1, 13)]  # k points de (1,1,1) a (12,12,12)
pseudopotenciales = ['Si.bhs', 'Si.pbe-nl-rrkjus_psl.1.0.0', 'Si.rel-pbe-rrkj']     

### Parámetros que cambian de elemento a elemento
Restringimos a FCC que es ibrav 2, nat = 1, peso , atomic positions en angstrongs

In [28]:
plantilla = """
&control
  calculation='scf',
  prefix='{elemento}',
  tstress = .true.,
  tprnfor = .true.,
  outdir='./out_{elemento}',
  pseudo_dir='/q-w-qe-6.3/pseudo',
/
&system
  ibrav=2,
  A = 3.6,
  nat=2, 
  ntyp=1,
  ecutwfc={ecut},
/
&electrons
  diagonalization='david',
  conv_thr=1.0d-7,
/
ATOMIC_SPECIES
  {elemento}  28.085  {pseudopotencial}.UPF
ATOMIC_POSITIONS {{crystal}}
  {elemento} 0.00 0.00 0.00 
  {elemento} 0.25 0.25 0.25
K_POINTS {{automatic}}
  {kx} {ky} {kz} 0 0 0
"""

In [29]:
# Nombre del archivo CSV actualizado para generarlo en el mismo bucle
nombre_csv = os.path.join(directorio, "SiData.csv")

In [30]:
# Abrir el archivo CSV para escribir
with open(nombre_csv, mode='w', newline='') as csv_file:
    fieldnames = ['Archivo', 'Elemento', 'Ecut', 'KPoints', 'Pseudopotencial']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
    # Generar archivos .in y registrarlos en el CSV
    for ecut, kpoints, pseudopotencial in itertools.product(ecuts, kpuntos, pseudopotenciales):
        kx, ky, kz = kpoints
        nombre_archivo = f"{elemento}_ecut{ecut}_k{kx}{ky}{kz}_{pseudopotencial}.in"
        path_completo = os.path.join(directorio, nombre_archivo)
        
        with open(path_completo, 'w') as archivo:
            contenido = plantilla.format(
                elemento=elemento,
                ecut=ecut,
                kx=kx, ky=ky, kz=kz,
                pseudopotencial=pseudopotencial,
                pos=pos
            )
            archivo.write(contenido)
        
        # Registrar en el CSV
        writer.writerow({
            'Archivo': nombre_archivo,
            'Elemento': elemento,
            'Ecut': ecut,
            'KPoints': f"{kx} {ky} {kz}",
            'Pseudopotencial': pseudopotencial
        })